# Part 1 Data Scraping!

   ## Scrape data by Xpath, then clean the unexpected elements
  

In [1]:
# !pip install lxml


In [2]:
#1.Scraping data from Wiki Pedia
    
#1.1 import required modules
import requests
from lxml import etree
import pandas as pd

#1.2 set url and headers
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

headers = {
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
            }
    
#1.3 get html strings
response = requests.get(url,headers = headers)
content = response.content.decode("utf8")

#1.4 etree parse
html = etree.HTML(content)





In [3]:
#1.5 Xpath extract data

trs = html.xpath("//div[1]/table[1]/tbody/tr")
#an empty list to input the dic-s
dic_list = []
for tr in trs:
    temp_dic = {}#temp dic for put values
    tds = tr.xpath("./td")
    for td in tds:
        Postalcode = td.xpath("./p/text()[1]")
        Borough = td.xpath("./p/span/a[1]/text()")
        Neighborhood = td.xpath("./p/span/a/text()")[1:] # because the first item is the borough name
        
        
        
        #to convert list elements of Neighborhood into str seperated by, 
        temp_str = ""
        if Neighborhood != []:
            counter = len(Neighborhood)
            for each in Neighborhood:
                counter -= 1
                temp_str = temp_str + each 
                if counter > 1:
                    temp_str = temp_str +  ", "
            Neighborhood = temp_str
         #to convert list elements of Neighborhood into str seperated by, 
        
        
        
        temp_dic = {
                "Postalcode":Postalcode,
                "Borough":Borough,
                "Neighborhood":Neighborhood, #since we may have one more items in the list 
                }
        dic_list.append(temp_dic)

dic_list

[{'Postalcode': ['M1A'], 'Borough': [], 'Neighborhood': []},
 {'Postalcode': ['M2A'], 'Borough': [], 'Neighborhood': []},
 {'Postalcode': ['M3A'],
  'Borough': ['North York'],
  'Neighborhood': 'Parkwoods'},
 {'Postalcode': ['M4A'],
  'Borough': ['North York'],
  'Neighborhood': 'Victoria Village'},
 {'Postalcode': ['M5A'],
  'Borough': ['Downtown Toronto'],
  'Neighborhood': 'Regent ParkHarbourfront'},
 {'Postalcode': ['M6A'],
  'Borough': ['North York'],
  'Neighborhood': 'Lawrence ManorLawrence Heights'},
 {'Postalcode': ['M7A'], 'Borough': ["Queen's Park"], 'Neighborhood': []},
 {'Postalcode': ['M8A'], 'Borough': [], 'Neighborhood': []},
 {'Postalcode': ['M9A'],
  'Borough': ['Etobicoke'],
  'Neighborhood': 'Islington Avenue'},
 {'Postalcode': ['M1B'],
  'Borough': ['Scarborough'],
  'Neighborhood': 'MalvernRouge'},
 {'Postalcode': ['M2B'], 'Borough': [], 'Neighborhood': []},
 {'Postalcode': ['M3B'],
  'Borough': ['North York'],
  'Neighborhood': 'Don Mills'},
 {'Postalcode': ['M4B

In [4]:
# creat a new Dataframe
columns= ['Postalcode','Borough','Neighborhood']
df_toronto = pd.DataFrame(columns = columns,data = dic_list)
df_toronto

,Postalcode,Borough,Neighborhood
0,[M1A],[],[]
1,[M2A],[],[]
2,[M3A],[North York],Parkwoods
3,[M4A],[North York],Victoria Village
4,[M5A],[Downtown Toronto],Regent ParkHarbourfront
...,...,...,...
175,[M5Z],[],[]
176,[M6Z],[],[]
177,[M7Z],[],[]
178,[M8Z],[Etobicoke],MimicoThe Queensway


In [5]:
type(df_toronto["Borough"][0])

list

In [6]:
#get the value from the [list]
df_toronto["Postalcode"] = df_toronto["Postalcode"].apply(lambda x:x[0]) 
#get the value from the [list] and make all the [] into ""
df_toronto["Borough"] = df_toronto["Borough"].apply(lambda x:x[0]if x != [] else "") 
#make all the [] into ""
df_toronto["Neighborhood"] = df_toronto["Neighborhood"].apply(lambda x:x if x != [] else "")


In [8]:
#clean all the empty string: ""
df_toronto = df_toronto[df_toronto["Borough"]!= ""]
df_toronto = df_toronto[df_toronto["Neighborhood"]!= ""]

In [9]:
df_toronto = df_toronto.reset_index(drop = True)
df_toronto.shape

(87, 3)

In [10]:
df_toronto

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent ParkHarbourfront
3,M6A,North York,Lawrence ManorLawrence Heights
4,M9A,Etobicoke,Islington Avenue
...,...,...,...
82,M5X,Downtown Toronto,First Canadian PlaceUnderground city
83,M8X,Etobicoke,The KingswayOld Mill
84,M4Y,Downtown Toronto,Church and Wellesley
85,M8Y,Etobicoke,"Old Mill, Sunnylea, Humber Bay, MimicoThe Quee..."


In [11]:
# df_toronto.to_csv("df_toronto.csv")

# Part 2 Making DataFrame Containing Coordinate Data

## install geocoder to quickly get the coordinate data

In [12]:
# !pip install geocoder

In [13]:
import geocoder #free and faster API to get coordinates data

# define the function get_coords to get latt 
def get_coords(postal_code):

    coords = None

    while(coords == None):

        geocoder_1 = geocoder.arcgis('%s, Toronto, Ontario'%format(postal_code))

        coords = geocoder_1.latlng

    return coords
#reference:https://www.coursera.org/learn/applied-data-science-capstone/discussions/all/threads/GUbn5pVFEemtYQ4WTQex_A

In [54]:
#set the list for catch temp_dic 
coords_list = []
Postalcodes_list = df_toronto["Postalcode"].tolist()
# print(Postalcodes_list)

for each in Postalcodes_list:
#     print(each)
    temp_dic = {}
    temp_coord = get_coords(each) 
    longitude = temp_coord[1]
    latitude = temp_coord[0]
    temp_dic = {
                "Postalcode":each,
                "latitude":latitude,
                "longitude":longitude,
                }
    coords_list.append(temp_dic)
#     break

df_toronto_coords = pd.DataFrame(coords_list)
print("finished!!!!!!!!!")

finished!!!!!!!!!


In [55]:
df_toronto_coords

,Postalcode,latitude,longitude
0,M3A,43.75245,-79.32991
1,M4A,43.73057,-79.31306
2,M5A,43.65512,-79.36264
3,M6A,43.72327,-79.45042
4,M9A,43.66263,-79.52831
5,M1B,43.81139,-79.19662
6,M3B,43.74923,-79.36186
7,M4B,43.70718,-79.31192
8,M5B,43.65739,-79.37804
9,M9B,43.65034,-79.55362


In [56]:
df_merge = pd.merge(df_toronto,df_toronto_coords, on = "Postalcode")
df_merge
# df_merge.to_csv("week3_df_merge.csv")

,Postalcode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,Regent ParkHarbourfront,43.65512,-79.36264
3,M6A,North York,Lawrence ManorLawrence Heights,43.72327,-79.45042
4,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831
5,M1B,Scarborough,MalvernRouge,43.81139,-79.19662
6,M3B,North York,Don Mills,43.74923,-79.36186
7,M4B,East York,Parkview HillWoodbine Gardens,43.70718,-79.31192
8,M5B,Downtown Toronto,Garden DistrictRyerson,43.65739,-79.37804
9,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.65034,-79.55362


In [57]:
# show the basic information of Toronto
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_merge['Borough'].unique()),
        df_merge.shape[0]
    )
)

The dataframe has 17 boroughs and 87 neighborhoods.


## Part3 Explore and Cluster the Neighborhoods 

In [58]:
#install required packages
!pip install geopy
!pip install folium==0.5.0

In [59]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [60]:
#get the coordinates inf of Toronto
geocoder_To = geocoder.arcgis('Toronto, Ontario')

coords_To = geocoder_To.latlng
coords_To

[43.648690000000045, -79.38543999999996]

## Create a map of Toronto with neighborhoods superimposed on top.

In [63]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[coords_To[0], coords_To[1]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['latitude'], df_merge['longitude'], df_merge['Borough'], df_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
                        ).add_to(map_Toronto) 

map_Toronto

 ## Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

## For this lab, I will focus on the Toronto Downtown Borough

In [72]:
downtown_df = df_merge[df_merge['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()

,Postalcode,Borough,Neighborhood,latitude,longitude
0,M5A,Downtown Toronto,Regent ParkHarbourfront,43.65512,-79.36264
1,M5B,Downtown Toronto,Garden DistrictRyerson,43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M5G,Downtown Toronto,Bay Street,43.65609,-79.38493
4,M5H,Downtown Toronto,RichmondKing,43.64970,-79.38258


In [70]:
#Let's get the geographical coordinates of Downtown Toronto
address ='Toronto, Ontario'
geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6534817, -79.3839347.


In [73]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_df['latitude'], downtown_df['longitude'], downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

## Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [74]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'TMSOIBPYAEZH2I111PMRWAMLGYGBGGTW2V5PJRGKHXL14JEH' # your Foursquare ID
CLIENT_SECRET = '420MGMXY1GS0DXLCXKGGGA3OBFHMP1RJFH1BIXBYRC4YPKW2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TMSOIBPYAEZH2I111PMRWAMLGYGBGGTW2V5PJRGKHXL14JEH
CLIENT_SECRET:420MGMXY1GS0DXLCXKGGGA3OBFHMP1RJFH1BIXBYRC4YPKW2


In [75]:
downtown_df.loc[0, 'Neighborhood']

'Regent ParkHarbourfront'

In [76]:
neighborhood_latitude = downtown_df.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_df.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = downtown_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent ParkHarbourfront are 43.65512000000007, -79.36263999999994.


## First, let's create the GET request URL. Name your URL url.

In [77]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius /m

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TMSOIBPYAEZH2I111PMRWAMLGYGBGGTW2V5PJRGKHXL14JEH&client_secret=420MGMXY1GS0DXLCXKGGGA3OBFHMP1RJFH1BIXBYRC4YPKW2&v=20180605&ll=43.65512000000007,-79.36263999999994&radius=500&limit=100'

In [81]:
results = requests.get(url).json()
# results

## From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [86]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# get the result for Foursquare API
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns #read for loop first
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


<ipython-input-86-6b7e76ab8bb2>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Berkeley Church,Event Space,43.655123,-79.365873


In [87]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown Toronto

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Now write the code to run the above function on each neighborhood and create a new dataframe called downtown_venues.

In [100]:
# type your answer here
names = downtown_df['Neighborhood']
latitudes = downtown_df['latitude']
longitudes = downtown_df['longitude']

downtown_venues = getNearbyVenues(names, latitudes, longitudes, radius=500)


Regent ParkHarbourfront
Garden DistrictRyerson
St. James Town
Bay Street
RichmondKing
Harbourfront, Union StationToronto Islands
Toronto Dominion CentreDesign Exchange
Commerce CourtVictoria Hotel
University of Toronto
Kensington Market, ChinatownGrange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South NiagaraIsland airport
Rosedale
St. James TownCabbagetown
First Canadian PlaceUnderground city
Church and Wellesley


In [101]:
print(downtown_venues.shape)
downtown_venues.head()

(1034, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent ParkHarbourfront,43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent ParkHarbourfront,43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent ParkHarbourfront,43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Regent ParkHarbourfront,43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,Regent ParkHarbourfront,43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space


In [102]:
#check for how many venues were returned for each neighborhoody 
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,71,71,71,71,71,71
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South NiagaraIsland airport",78,78,78,78,78,78
Church and Wellesley,81,81,81,81,81,81
Commerce CourtVictoria Hotel,100,100,100,100,100,100
First Canadian PlaceUnderground city,100,100,100,100,100,100
Garden DistrictRyerson,100,100,100,100,100,100
"Harbourfront, Union StationToronto Islands",53,53,53,53,53,53
"Kensington Market, ChinatownGrange Park",49,49,49,49,49,49
Regent ParkHarbourfront,21,21,21,21,21,21


In [103]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 178 uniques categories.


#  Analyze Each Neighborhood

In [106]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")
downtown_onehot.head()


,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [107]:

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent ParkHarbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent ParkHarbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent ParkHarbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent ParkHarbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [109]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index() #requency of occurrence of each category
downtown_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Bay Street,0.000000,0.000000,0.0,0.014085,0.014085,0.0,0.000000,0.0,0.000000,0.014085,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.014085,0.0,0.014085,0.000000,0.028169,0.00,0.014085,0.000000,0.0,0.014085,0.0,0.000000,0.0,0.0,0.098592,0.000000,0.112676,0.0,0.000000,0.0,0.0,0.0,0.014085,0.00,0.028169,0.000000,0.000000,0.00,0.014085,0.014085,0.014085,0.000000,0.000000,0.000000,0.0,0.014085,0.000000,0.0,0.0,0.000000,0.0,0.0,0.014085,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.014085,0.0,0.0,0.000000,0.014085,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.014085,0.000000,0.0,0.000000,0.0,0.028169,0.000000,0.0,0.000000,0.000000,0.000000,0.014085,0.014085,0.00,0.0,0.014085,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.028169,0.014085,0.014085,0.00,0.00,0.0,0.014085,0.00,0.014085,0.014085,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.014085,0.000000,0.0,0.000000,0.014085,0.0,0.028169,0.014085,0.0,0.014085,0.000000,0.0,0.014085,0.0,0.000000,0.000000,0.000000,0.028169,0.000000,0.014085,0.014085,0.014085,0.000000,0.0,0.028169,0.000000,0.00,0.0,0.000000,0.0,0.014085,0.000000,0.0,0.028169,0.0,0.00,0.0,0.014085,0.014085,0.000000,0.014085,0.000000,0.000000,0.00,0.014085,0.014085,0.00,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.012821,0.000000,0.0,0.000000,0.000000,0.0,0.012821,0.0,0.025641,0.012821,0.038462,0.0,0.012821,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.012821,0.000000,0.00,0.012821,0.000000,0.

## Let's print each neighborhood along with the top 5 most common venues

In [112]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.10
2             Spa  0.03
3  Sandwich Place  0.03
4           Hotel  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South NiagaraIsland airport----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.06
2                Café  0.05
3                Park  0.04
4   French Restaurant  0.04


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese Restaurant  0.06
2           Restaurant  0.05
3     Sushi Restaurant  0.05
4              Gay Bar  0.04


----Commerce CourtVictoria Hotel----
                 venue  freq
0          Coffee Shop  0.10
1           Restaurant  0.08
2                Hotel  0.06
3                 Café  0.05
4  Japanese Restaurant  0.04


----First Canadian PlaceUnderground city----
         venue  freq
0  Coffee Shop  0.12
1        Hotel  0.07
2         Café  0.06
3          Gym  0.04
4   

## Let's put that into a _pandas_ dataframe

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [129]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Clothing Store,Spa,Sandwich Place,Hotel,Bubble Tea Shop,Sushi Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Plaza
1,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Park,French Restaurant,Gym / Fitness Center,Bar,Grocery Store,Sandwich Place,Bakery
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Fast Food Restaurant,Grocery Store,Mediterranean Restaurant,Men's Store,Dance Studio
3,Commerce CourtVictoria Hotel,Coffee Shop,Restaurant,Hotel,Café,Japanese Restaurant,Italian Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar
4,First Canadian PlaceUnderground city,Coffee Shop,Hotel,Café,Gym,Restaurant,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Asian Restaurant


## Cluster Neighborhoods

In [130]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 3, 1, 1, 1, 4, 1, 3, 2, 1])

In [134]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent ParkHarbourfront,43.65512,-79.36264,2,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Electronics Store,Event Space,Pub,Restaurant,Food Truck,Spa
1,M5B,Downtown Toronto,Garden DistrictRyerson,43.65739,-79.37804,4,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Hotel,Japanese Restaurant,Italian Restaurant,Department Store,Diner,Bookstore
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,4,Coffee Shop,Cosmetics Shop,Clothing Store,Cocktail Bar,Italian Restaurant,Café,Restaurant,Hotel,Theater,Seafood Restaurant
3,M5G,Downtown Toronto,Bay Street,43.65609,-79.38493,4,Coffee Shop,Clothing Store,Spa,Sandwich Place,Hotel,Bubble Tea Shop,Sushi Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Plaza
4,M5H,Downtown Toronto,RichmondKing,43.64970,-79.38258,1,Café,Hotel,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Salad Place,Steakhouse,Asian Restaurant,Bakery


In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
#classify the data based on 'Cluster Labels'
for lat, lon, poi, cluster in zip(downtown_merged['latitude'], downtown_merged['longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    #plot different venue in different color based on different Cluster Labels
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [137]:
#Cluster 1
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,0,Park,Playground,Bike Trail,Nightclub,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum


In [138]:
#Cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,1,Café,Hotel,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Salad Place,Steakhouse,Asian Restaurant,Bakery
5,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Hotel,Park,Café,Plaza,Boat or Ferry,Restaurant,Shopping Mall,Sports Bar
6,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Salad Place,Bakery,Seafood Restaurant,Sushi Restaurant
7,Downtown Toronto,1,Coffee Shop,Restaurant,Hotel,Café,Japanese Restaurant,Italian Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar
13,Downtown Toronto,1,Coffee Shop,Hotel,Café,Gym,Restaurant,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Asian Restaurant
14,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Fast Food Restaurant,Grocery Store,Mediterranean Restaurant,Men's Store,Dance Studio


In [139]:
#Cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Electronics Store,Event Space,Pub,Restaurant,Food Truck,Spa


In [140]:
#Cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,3,Café,Bookstore,Bakery,Coffee Shop,Japanese Restaurant,Restaurant,Gym,Bar,Fish & Chips Shop,Grocery Store
9,Downtown Toronto,3,Café,Caribbean Restaurant,Grocery Store,Vietnamese Restaurant,Park,Vegetarian / Vegan Restaurant,Bakery,Dumpling Restaurant,Bar,Coffee Shop
10,Downtown Toronto,3,Italian Restaurant,Coffee Shop,Café,Park,French Restaurant,Gym / Fitness Center,Bar,Grocery Store,Sandwich Place,Bakery
12,Downtown Toronto,3,Park,Coffee Shop,Pizza Place,Italian Restaurant,Restaurant,Bakery,Pub,Café,Pharmacy,Chinese Restaurant
